In [1]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext

In [2]:
# [+] PySpark 임포트 및 설정
conf = SparkConf().setMaster("local").setAppName("key-value-rdd-operations")
sc = SparkContext(conf = conf)

### groupBy(): 입력 함수를 기준으로 그룹핑

In [3]:
# [+] 리스트로부터 rdd 생성 [1, 1, 2, 3, 5, 8]
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])

In [4]:
# [+] groupBy()를 이용한 홀수/짝수 그룹핑 -> groups
groups = rdd.groupBy(lambda x : x % 2)

In [6]:
# [+] 그룹핑 결과를 리스트로 반환 -> res
res = groups.collect()

[(1, <pyspark.resultiterable.ResultIterable at 0x1f7a6e10e80>),
 (0, <pyspark.resultiterable.ResultIterable at 0x1f7a87b3fa0>)]

In [7]:
# [+] 그룹핑 결과(res) 출력
for k, v in res:
    print(k, list(v))

1 [1, 1, 3, 5]
0 [2, 8]


### groupByKey(): Key 값을 기준으로 그룹핑

In [ ]:
# [+] 리스트로부터 key-value rdd 생성 [('a', 1), ('b', 1), ('a', 1)]
rdd = sc.parallelize([('a', 1), ('b', 1), ('a', 1)])
rdd.collect()

In [ ]:
# [+] groupByKey()를 이용한 그룹핑 및 리스트 반환 -> groups
groups = rdd.groupByKey()

In [ ]:
# [+] 그룹핑 결과를 리스트로 반환 -> res
res = groups.collect()
res

In [ ]:
# [+]그룹핑 결과(res) 출력
for k, v in res:
    print(k, list(v))

In [ ]:
# len(): Python 객체의 길이를 리턴
value = [1, 1]
len(value)

In [ ]:
# mapValues(), len()을 이용한 값 빈도 계산
counts = groups.mapValues(len)

In [ ]:
counts.collect()

### groupBy() 예제
1. 리스트로부터 RDD 객체 생성
    + 리스트: ['C', 'C++', 'Python', 'Java', 'C#']  
2. 원소의 첫 번째 문자 값을 기준으로 그룹핑
    + 기대 결과:  
        J ['Java']  
        C ['C', 'C++', 'C#']  
        P ['Python']  
3. 그룹핑 결과를 리스트 객체로 출력

In [ ]:
# [+] 리스트로부터 RDD 객체 생성
rdd = sc.parallelize(['C', 'C++', 'Python', 'Java', 'C#'])

In [ ]:
# [+] 원소별 첫 번째 값(x[0])을 기준으로 그룹핑 -> groups
groups = rdd.groupBy(lambda x:x[0])

In [ ]:
# [+] 그룹핑 결과를 리스트로 반환 -> res
res = groups.collect()

In [ ]:
# [+] 그룹핑 결과(res) 출력
for k, v in res:
    print(k, list(v))

### groupByKey() 예제
1. list 객체로부터 3개의 파티션을 갖는 Key-Value RDD 생성하기
    + getNumPatitions(): RDD 객체의 파티션 수 출력
2. Key 별로 그룹핑
3. 그룹핑 결과 리스트로 반환
4. 리스트 출력

In [ ]:
# list 객체로부터 3개의 파티션을 갖는 Key-Value RDD 생성하기

rdd = sc.parallelize([  # 과목 별 점수 데이터
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3) # 파티션 수: 3

In [3]:
# getNumPartitions(): RDD 객체의 파티션 수를 확인하는 액션 메서드
rdd.getNumpartitions()

NameError: name 'rdd' is not defined

In [22]:
# [+] Key 별로 그룹핑 -> groups
groups = rdd.groupByKey()

In [23]:
# [+] 그룹핑 결과 리스트로 반환 -> res
res = groups.collect()

In [24]:
# [+] 그룹핑 결과(res) 출력
for k, v in res:
    print(k, list(v))

Science [7, 6, 9, 5]
Math [7, 2, 8, 3]
English [7, 4, 9, 4]


In [26]:
# key별 그룹핑을 2개의 파티션으로 분리하여 수행 -> groups
groups = rdd.groupByKey(2)

In [27]:
# [+] 파티션 수 출력
groups.getNumPartitions()

2

In [33]:
# 파티션별 RDD 값들을 리스트로 반환
res = groups.glom().collect()

In [29]:
#for i, partition in enumerate(res):
 #   print(f"partition{i} : {partition}\n")

partition0 : ('Science', <pyspark.resultiterable.ResultIterable object at 0x000002685256C4C0>)

partition1 : ('Math', <pyspark.resultiterable.ResultIterable object at 0x0000026853FC9B80>)

partition2 : ('English', <pyspark.resultiterable.ResultIterable object at 0x0000026853FC9880>)



In [34]:
res

[[('English', <pyspark.resultiterable.ResultIterable at 0x26853fc98b0>),
  ('Science', <pyspark.resultiterable.ResultIterable at 0x26853e7a7c0>)],
 [('Math', <pyspark.resultiterable.ResultIterable at 0x26853f55c40>)]]

In [35]:
# [+] 0번째 파티션에 대한 그룹핑 결과(res)를 출력
for k, v in res[0]:
    print(k, list(v))

English [7, 4, 9, 4]
Science [7, 6, 9, 5]


In [36]:
# [+] 1번째 파티션에 대한 그룹핑 결과(res)를 출력
for k, v in res[1]:
    print(k, list(v))

Math [7, 2, 8, 3]


### reduce() vs reduceByKey()
+ reduce(): 입력 함수를 기준으로 집계를 수행하는 액션 메서드
+ reduceByKey(): Key를 기준으로 그룹핑 및 집계를 수행하는 변환 메서드

In [4]:
# [+] reduce() 를 이용한 총합 구하기
sc.parallelize([1, 2, 3, 4, 5]).reduce(lambda x, y : x + y)

15

In [47]:
# [+] 리스트로부터 K-V RDD 생성 -> rdd
rdd = sc.parallelize(['a', 'a', 'a', 'b', 'b', 'c', 'c'])
kvrdd = rdd.map(lambda x : (x, 1))
kvrdd.collect()

[('a', 1), ('a', 1), ('a', 1), ('b', 1), ('b', 1), ('c', 1), ('c', 1)]

In [52]:
# [+] reduceByKey()를 이용한 Key별 총합 계산 및 리스트로 반환
reduced = kvrdd.reduceByKey(lambda a, b : a + b)
reduced.collect()

[('a', 3), ('b', 2), ('c', 2)]

In [6]:
# 과목 별 점수 데이터
rdd = sc.parallelize([
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3)

In [65]:
# [+] reduceByKey() 를 이용한 과목 별 점수 총합 구하기
scores = rdd.reduceByKey(lambda x, y: x + y)

[('Science', 27), ('Math', 20), ('English', 24)]

In [11]:
# 과목별 총점과 평균도 구해보기
scores_transformed = rdd.mapValues(lambda x : (x, 1))
scores_transformed.collect()

[('Math', (7, 1)),
 ('Math', (2, 1)),
 ('English', (7, 1)),
 ('Science', (7, 1)),
 ('English', (4, 1)),
 ('English', (9, 1)),
 ('Math', (8, 1)),
 ('Math', (3, 1)),
 ('English', (4, 1)),
 ('Science', (6, 1)),
 ('Science', (9, 1)),
 ('Science', (5, 1))]

In [13]:
scores_reduced = scores_transformed.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
scores_reduced.collect()

[('Science', (27, 4)), ('Math', (20, 4)), ('English', (24, 4))]

### mapValues() vs map()
+ mapValues(): value에만 함수를 적용

In [73]:
# 리스트로부터 K-V RDD 객체 생성
rdd = sc.parallelize([
    ('a', ['apple', 'banana', 'lemon']),
    ('b', ['grapes'])
])

In [74]:
# mapValues(), len()를 이용하여 키 별 갯수 세기
rdd.mapValues(len).collect()

[('a', 3), ('b', 1)]

In [75]:
# map(): key, value 에 함수를 적용
rdd.map(len).collect()

[2, 2]

### countByKey()
Key별 값 개수를 계산 및 딕셔너리로 반환하는 액션 메서드

In [76]:
# countByKey()를 이용하여 Key별 값 배수 계산 및 반환 -> res
rdd = sc.parallelize([
    ('a', 1), ('b', 1), ('a', 1)
])

res = rdd.countByKey()

In [77]:
# countByKey() 결과 출력
print(res)

defaultdict(<class 'int'>, {'a': 2, 'b': 1})


### keys()
key 목록을 RDD로 생성하는 변환 메서드

In [14]:
# 과목 별 점수 데이터
rdd = sc.parallelize([
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3)

In [82]:
# RDD 키 목록 생성 및 출력
rdd_keys = rdd.keys()
rdd_keys.collect()

['Math',
 'Math',
 'English',
 'Science',
 'English',
 'English',
 'Math',
 'Math',
 'English',
 'Science',
 'Science',
 'Science']

In [16]:
# [+] 유니크한 키 값 출력
rdd.keys().distinct().collect()

['Science', 'Math', 'English']

In [17]:
# [+] 유니크한 키 개수 출력
rdd.keys().distinct().count()

3

### Join 연산
+ join(): 내부조인
+ leftOuterJoin(): 외부조인(첫 번째 RDD 중심)
+ rightOuterJoin(): 외부조인(두 번째 RDD 중심)

In [91]:
# K-V RDD 객체 두 개 생성 
rdd1 = sc.parallelize([("a", 1), ("b", 4)])
rdd2 = sc.parallelize([("a", 2)])

In [93]:
# Join 연산
rdd1.join(rdd2).collect()

[('a', (1, 2))]

In [94]:
# Left Outer Join 연산
rdd1.leftOuterJoin(rdd2).collect()

[('b', (4, None)), ('a', (1, 2))]

In [96]:
# Right Outer Join 연산
rdd1.rightOuterJoin(rdd2).collect()

[('a', (1, 2))]